Overriding virtual functions in Python
Suppose that a C++ class or interface has a virtual function that we’d like to override from within Python as:

In [ ]:
/* C++ */
class Animal {
public:
    virtual ~Animal() { }
    virtual std::string go(int n_times) = 0;
};

class Dog : public Animal {
public:
    std::string go(int n_times) override {
        std::string result;
        for (int i=0; i<n_times; ++i)
            result += "woof! ";
        return result;
    }
};
/* A plain function which calls the function go() on arbitrary Animal instance */
std::string call_go(Animal *animal) {
    return animal->go(3);
}

Defining a new type of Animal from within Python is possible but requires a helper class that is defined as follows

In [ ]:
class PyAnimal : public Animal {
public:
    /* Inherit the constructors */
    using Animal::Animal;

    /* Trampoline (need one for each virtual function) */
    std::string go(int n_times) override {
        PYBIND11_OVERRIDE_PURE(
            std::string, /* Return type */
            Animal,      /* Parent class */
            go,          /* Name of function in C++ (must match Python name) */
            n_times      /* Argument(s) */
        );
    }
};

The macro PYBIND11_OVERRIDE_PURE should be used for pure virtual functions, and PYBIND11_OVERRIDE should be used for functions which have a default implementation. 
There are also two alternate macros PYBIND11_OVERRIDE_PURE_NAME and PYBIND11_OVERRIDE_NAME which take a string-valued name argument between the Parent class and Name of the function slots, which defines the name of function in Python. 
This is required when the C++ and Python versions of the function have different names, e.g. operator() vs __call__.

In [ ]:
PYBIND11_MODULE(example, m) {
    py::class_<Animal, PyAnimal>(m, "Animal")
        .def(py::init<>())
        .def("go", &Animal::go); //Bindings should be made against the actual class, not the trampoline helper class.

    py::class_<Dog, Animal>(m, "Dog")
        .def(py::init<>());

    m.def("call_go", &call_go);
}

If you are defining a custom constructor in a derived Python class, you must ensure that you explicitly call the bound C++ constructor using __init__, regardless of whether it is a default constructor or not. Otherwise, the memory for the C++ portion of the instance will be left uninitialized, which will generally leave the C++ instance in an invalid state and cause undefined behavior if the C++ instance is subsequently used.

Combining virtual functions and inheritance
When combining virtual methods with inheritance, you need to be sure to provide an override for each method for which you want to allow overrides from derived python classes.

In [ ]:
/* C++ */
class Animal {
public:
    virtual std::string go(int n_times) = 0;
    virtual std::string name() { return "unknown"; }
};
class Dog : public Animal {
public:
    std::string go(int n_times) override {
        std::string result;
        for (int i=0; i<n_times; ++i)
            result += bark() + " ";
        return result;
    }
    virtual std::string bark() { return "woof!"; }
};

/* Pybind11 */
class PyAnimal : public Animal {
public:
    using Animal::Animal; // Inherit constructors
    std::string go(int n_times) override { PYBIND11_OVERRIDE_PURE(std::string, Animal, go, n_times); }
    std::string name() override { PYBIND11_OVERRIDE(std::string, Animal, name, ); }
};
class PyDog : public Dog {
public:
    using Dog::Dog; // Inherit constructors
    std::string go(int n_times) override { PYBIND11_OVERRIDE(std::string, Dog, go, n_times); }
    std::string name() override { PYBIND11_OVERRIDE(std::string, Dog, name, ); }
    std::string bark() override { PYBIND11_OVERRIDE(std::string, Dog, bark, ); }
};

Here, the trampoline class for Animal must override go() and name(), but in order to allow python code to inherit properly from Dog, we also need a trampoline class for Dog that overrides both the added bark() method and the go() and name() methods inherited from Animal (even though Dog doesn’t directly override the name() method):

Protected member function, 
Normally, it's impossible to expose protected member functions to Python,

In [ ]:
/* C++ */
class A {
protected:
    int foo() const { return 42; }
};
/* Pybind11 */
py::class_<A>(m, "A")
    .def("foo", &A::foo); // error: 'foo' is a protected member of 'A'

To make use of protected functions in derived Python classes, one can do 

In [ ]:
/* C++ */
class Publicist : public A { // helper type for exposing protected functions
public:
    using A::foo; // inherited with different access modifier
};
/* Pybind11 */
py::class_<A>(m, "A") // bind the primary class
    .def("foo", &Publicist::foo); // expose protected methods via the publicist

This works because &Publicist::foo is exactly the same function as &A::foo (same signature and address), just with a different access modifier. The only purpose of the Publicist helper class is to make the function name public.
If the intent is to expose protected virtual functions which can be overridden in Python, the publicist pattern can be combined with the previously described trampoline

In [ ]:
/* C++ */
class A {
public:
    virtual ~A() = default;

protected:
    virtual int foo() const { return 42; }
};

class Trampoline : public A {
public:
    int foo() const override { PYBIND11_OVERRIDE(int, A, foo, ); }
};

class Publicist : public A {
public:
    using A::foo;
};
/* Pybind11 */
py::class_<A, Trampoline>(m, "A") // <-- `Trampoline` here
    .def("foo", &Publicist::foo); // <-- `Publicist` here, not `Trampoline`!

Class can not be inherited from. 
The py::is_final attribute can be used to ensure that Python classes cannot inherit from a specified type. The underlying C++ type does not need to be declared final.

In [ ]:
/* C++ */
class IsFinal final {};
/* Pybind11 */
py::class_<IsFinal>(m, "IsFinal", py::is_final());

Class with template parameter
As C++ templates may only be instantiated at compile time, so pybind11 can only wrap instantiated templated classes. 
One cannot wrap a non-instantiated template

In [ ]:
/* C++ */
struct Cat {};
struct Dog {};

template <typename PetType>
struct Cage {
    Cage(PetType& pet);
    PetType& get();
};
/* Pybind11 */
py::class_<Cage<Cat>>(m, "CatCage")
    .def("get", &Cage<Cat>::get);
    
py::class_<Cage<Dog>>(m, "DogCage")
    .def("get", &Cage<Dog>::get);

With template parameter in Class mehtod

In [ ]:
/* C++ */
typename <typename T>
struct MyClass {
    template <typename V>
    T fn(V v);
};
/* Pybind11 */
py::class<MyClass<int>>(m, "MyClassT")
    .def("fn", &MyClass<int>::fn<std::string>);